Created on 2023-08-21
This notebook ...

TODO: Notebook description

In [ ]:
## Library imports
import os
from utils import select_files, create_directory
from create_helper import create_fieldset, create_particleset, create_kernel
from plotting import plot_release_locations
from datetime import datetime, timedelta

import numpy as np

# User options
TODO: Write a description here

In [ ]:
## User options
user_settings = {'user': 'denes001',
                 'output_directory': '/storage/shared/oceanparcels/output_data/data_Michael/PlasticTransportKernels/data/',
                 'fig_directory': '/storage/shared/oceanparcels/output_data/data_Michael/PlasticTransportKernels/fig/',
                 }

## Create directories if missing
create_directory(user_settings['output_directory'])
create_directory(user_settings['fig_directory'])

# Model settings
TODO: Write a description here

In [ ]:
## Model settings
model_settings = {'input_data_dir': '/storage/shared/oceanparcels/input_data/',
                  'input_data_dir_2': '/storage/shared/oceanparcels/output_data/data_Michael/',
                  'input_data_dir_3': '/storage/shared/oceanparcels/output_data/data_Mikael/',

                  'mode': '3D',
                  
                  # Ocean model
                  'ocean_dir': 'MOi/psy4v3r1/',                 # Directory of ocean model data
                  'ocean_filename': 'psy4v3r1-daily_',          # Filename style of ocean model data
                  'ocean_mesh': 'MOi/domain_ORCA0083-N006/coordinates.nc',                               # File location of ocean mesh
                  'bathymetry_mesh': 'MOi/domain_ORCA0083-N006/bathymetry_ORCA12_V3.3.nc', # File location of bathymetry mesh
                  'mixing_f': True,                             # Turn on/off vertical turbulent mixing
                
                  # Biogeochemistry model
                  'biofouling_f': True,                         # Turn on/off biofouling of particles
                  'bgc_dir': 'MOi/biomer4v2r1/',                # Directory of biogeochemistry model
                  'bgc_filename': 'biomer4v2r1-weekly_',
                  'bgc_mesh': 'MOi/domain_ORCA025-N006/mesh_hgr_PSY4V3_deg.nc', # File location of biogeochemistry model mesh

                  'bgc_biofilm_density': 1388.,                 # Biofilm density [g m-3]
                  'bgc_algae_cell_volume': 2.0E-16,             # Volume of 1 algal cell [m3]
                  'bgc_boltzmann_constant': 1.0306E-13 / (86400. ** 2.),  # Boltzmann constant [m2 kg d-2 K-1] now [s-2] (=1.3804E-23)
                  'bgc_respiration_rate': 0.1 / 86400.,         # Respiration rate, now [s-1]
                  'bgc_respiration_temperature_coefficient': 2.13,# temperature coefficient respiration [-]
                  'bgc_shear_frequency': 1.728E5 / 86400.,      # Shear [d-1], now [s-1]
                  'bgc_carbon_atomic_weight': 12.,              # Atomic weight of Carbon
                  'bgc_collision_probability': 1.,              # Collision probability [-]
                  
                  # Waves model
                  'stokes_f': True,                             # Turn on/off Stokes Drift
                  'stokes_dir': 'ERA5/waves/',                  # Directory of Stokes drift model data
                  'stokes_filename': 'ERA5_global_waves_monthly_converted_',

                  # Wind model
                  'wind_f': True,                               # Turn on/off Windage
                  'wind_dir': 'ERA5/wind/',                     # Directory of Wind model data
                  'wind_filename': 'ERA5_global_wind_monthly_converted_',

                  'allow_time_extrapolation': False,            # Allow extrapolation of time for fieldset
                  'verbose_delete': False,                      # Print extra information when executing delete operations
                  }

particle_settings = {'start_date': datetime.strptime('2019-01-01-00:00:00', '%Y-%m-%d-%H:%M:%S'), # Start date of simulation
                     'runtime': timedelta(days=70),             # Runtime of simulation, use negative if releasing particles backwards in time
                     'dt_write': timedelta(days=1),             # Timestep of output
                     'dt_timestep': timedelta(minutes=5),       # Timestep of advection
                     # TODO: Could create own particle class with own sampling kernels to append after helper?
                     #'particle_class': PlasticParticle, # Particle class to use, feel free to create your own based on the base PlasticParticle class
                    }
fieldset = create_fieldset(model_settings, particle_settings)

# Particleset settings
TODO: Write description here -- user must define particle release locations

In [ ]:
## Set particle release locations
n_particles = 20
lons = np.linspace(-35,-10,n_particles)
lats = np.linspace(30,60,n_particles)
depths = np.random.uniform(0,1000, n_particles)
times = np.linspace(0,50000,n_particles)

release_locations = {'lons': lons,
                     'lats': lats,
                     'depths': depths,
                     'times': times
}   


## Show particle release locations
plot_release_locations(release_locations)

In [ ]:
# Other settings
wind_test = np.random.uniform(0,0.05,n_particles)
density_test = np.random.uniform(1000,1050,n_particles)
diameter_test = np.random.uniform(0.00001, 0.01, n_particles)
## Particleset settings
particle_settings.update({'release_locations': release_locations, # A dictionary of lons, lats, depths (optional), and times (optional)
                          'windage_coefficient': wind_test, # A float or Nx1 array of windage coefficients, values in [0,1] [percent of wind speed]
                          'particle_density': density_test, # A float or Nx1 array of particle densities [kg m-3]
                          'particle_diameter': diameter_test, # A float or Nx1 array of particle lengths [m]
                          'write_output_option': 'all' # Option to write variables. Options: 'none' (default), 'tracer' (saves tracer fields), 'dynamic' (saves all variables that change spatiotemporally), 'all' (saves all variables)
                           })


pset = create_particleset(fieldset, particle_settings)

# Kernel creation
This should be pretty simple based on the flags above

In [ ]:
kernels = create_kernel(fieldset, pset)
# Advection and execution



# Plotting, analysis, animations

